In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349500049
Segment 648518346349500049 vertex labels regenerated in: 0.2960648536682129 seconds.
648518346349500139
Segment 648518346349500139 vertex labels regenerated in: 0.27749061584472656 seconds.
648518346349500162
Segment 648518346349500162 vertex labels regenerated in: 0.2881183624267578 seconds.
648518346349500440
Segment 648518346349500440 vertex labels regenerated in: 0.31190943717956543 seconds.
648518346349500657
Segment 648518346349500657 vertex labels regenerated in: 0.33080053329467773 seconds.
648518346349500886
Segment 648518346349500886 vertex labels regenerated in: 0.2610037326812744 seconds.
648518346349501216
Segment 648518346349501216 vertex labels regenerated in: 0.3052222728729248 seconds.
648518346349501597
Segment 648518346349501597 vertex labels regenerated in: 0.2705991268157959 seconds.
648518346349501787
Segment 648518346349501787 vertex labels regenerated in: 0.7156996726989746 seconds.
648518346349503591
Segment 648518346349503591 vertex labels r

In [5]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()